# Importación de librerias

In [1]:
import os
import glob
import h5py as h5
from astropy.time import Time
import pandas as pd
import numpy as np
import sigpyproc
from sigpyproc.readers import FilReader

# Definición de funciones

In [2]:
#Funcion para extraer información de archivos .hdf5 que da SpS
#Esta función solo depende del camino que le pases así que funcionara
#ya sea para el archivo .hdf5 que tenga la opción no filter, así como para
#aquel que no la tenga. Le pasas el path, y te devuelve un dataframe con
#toda la información de los candidatos, uno con toda la información de 
#los pulsos y otro con toda la información de los eventos

def get_hdf5_out_df(camino):
    path_hdf5 = camino
    #Definimos los nombres de las columnas que vamos a levantar
    #NOTA: aqui le llamo a la variable 'Sigma' de single_pulse_search_py SN, pero no es una SN como se calcula habitualmente
    # La información sale del script 'single_pulse_search.py' mismo, donde describe lo siguiente:
    #    -- The definition of "sigma" used is possibly slightly different
    #   from that used in other codes for S/N:
    #       sigma = sum(signal-bkgd_level)/RMS/sqrt(boxcar_width)
    #   where the bkgd_level is typically 0 after detrending and RMS=1
    #   after normalization.  This definition has the advantage that
    #   you will get (basically) the same sigma for any pulse no
    #   matter how much the input time series has been downsampled as
    #   long as the pulse is still resolved.
    # Copyright Scott Ransom <sransom@nrao.edu>, 2015
    
    column_cand = ['DM', 'Period', 'Time', 'SN', 'Rank', 'N_pulses', 'main_cand']
    column_pulses = ['DM', 'SN', 'Time', 'Downfact', 'Time_org', 'N_events', 'Sample', 'Candidate', 'Rank']
    column_events = ['DM', 'SN', 'Time', 'Downfact', 'Time_org', 'Sample', 'Pulse']
    
    #Leemos el archivo .hfd5
    lect_hdf5 = h5.File(path_hdf5, 'r')

    #Levanto la informacion de los candidatos
    datacb0 = lect_hdf5["candidates/block0_values"][:]
    datacb1 = lect_hdf5["candidates/block1_values"][:]
    datacb2 = lect_hdf5["candidates/block2_values"][:]

    dc = np.hstack((datacb0, datacb1, datacb2))
    candidates = pd.DataFrame(dc, columns = column_cand)

    #Levanto la informacion de los pulsos
    datapb0 = lect_hdf5["pulses/block0_values"][:]
    datapb1 = lect_hdf5["pulses/block1_values"][:]
    datapb2 = lect_hdf5["pulses/block2_values"][:]
    datapb3 = lect_hdf5["pulses/block3_values"][:]

    dp = np.hstack((datapb0, datapb1, datapb2, datapb3))
    pulses = pd.DataFrame(dp, columns = column_pulses)

    #Levanto la informacion de los eventos
    dataeb0 = lect_hdf5["events/block0_values"][:]
    dataeb1 = lect_hdf5["events/block1_values"][:]
    dataeb2 = lect_hdf5["events/block2_values"][:]

    de = np.hstack((dataeb0, dataeb1, dataeb2))
    events = pd.DataFrame(de, columns = column_events)
    
    return candidates, pulses, events

In [3]:
# Funcion para levantar la información de un archivo singlepulse
# para armarme el dataframe que quiero usar, además, antes tengo 
# que extraer información del archivo .fil, como el mjdstart y pasarselo

def get_pulsespresto_out_df(camino, mjdstart):
    #Array que tiene toda la información de dentro de un singlepulse
    #por algún motivo genial, las líneas del archivo que tienen un # adelante, no las levanta
    data = np.loadtxt(camino)

    #Para recopilar la información de la fecha del archivo la extraigo del nombre
    # para que esto funcione, tiene que tener la siguiente forma el nombre
    # del archivo singlepulse: '/all_sp_t_8SN_J1810-197_A2_20221231.singlepulse'
    # porque levanta la cadena de string que esta luego del ultimo / y luego del
    # ultimo _ y antes del .
    # Date from path:
    date_fpath = camino.split("/")[-1].split("_")[-1].split(".")[0]
    array_date = np.full(data.shape[0], np.int32(date_fpath))

    #Armo un array que tenga la información del tstart para este día
    array_mjdstart = np.full(data.shape[0], mjdstart)

    #Concateno todos los arrays con la información de la fehca y el mjd de comienzo de la observación
    array_pulses = np.concatenate((data, array_date.reshape(-1,1), array_mjdstart.reshape(-1,1)),axis=1)
    #Creo un dataframe
    column_pulsespresto = ['DM', 'SN', 'Time', 'Sample','Downsample', 'Date', 'MJD start']
    pulsespresto = pd.DataFrame(array_pulses, columns = column_pulsespresto)
    
    return pulsespresto
    
    

In [4]:
def pulsespresto_filter(dataframe, dmmin, dmmax):
    #Primer criterio para agrupar pulsos que voy a considerar son el mismo
    #Este criterio va a ser, truncar el tiempo en el que ocurre el pulso
    # y si es igual hasta la centesima, los agrupos. Luego, me quedo
    # con aquel el pulso para el cual la SN es maxima dentro de los que
    # tienen el mismo tiempo truncado. Y esos son los que "considero pulsos"
    dataframe['Time trunc']=np.trunc(dataframe['Time']*100)/100
    criterio = dataframe.groupby('Time trunc')['SN'].max()
    dataframe_filtered1 = dataframe[dataframe.apply(lambda row: row['SN'] == criterio[row['Time trunc']],axis=1)]

    #Luego, el otro filtro que aplico, es que el DM para el cual tenga la máxima
    # SN, sea +-10 alrededor del Dm del objeto, esto en realidad lo puedo especificar
    #en los parametros que le paso a esta funcion
    result_presto_sps = dataframe_filtered1[(dataframe_filtered1.DM > dmmin) & (dataframe_filtered1.DM< dmmax)]

    
    return result_presto_sps

In [5]:
def filter_fordm(dataframe, dmmin, dmmax):
    #el objetivo de esta función, es filtrar por dm cualquier dataframe, 
    #esta pensando en realidad para filtrar los dataframe que salen de los
    #archivos .hdf5, pero si por algun motivo no funciono el filtro
    #en pulsespresot_filter, podría usarse también. 

    dataframe_filtered = dataframe[(dataframe.DM > dmmin) & (dataframe.DM <dmmax)]
    return dataframe_filtered

In [6]:
def compare_presto_sps(dfpresto: pd.DataFrame, dfcand: pd.DataFrame, dfpuls: pd.DataFrame, spsoption: str):
    datatocompare = dfpresto[['DM','SN','Time', 'Sample','Date', 'MJD start']].copy()
    cols_to_cand = ['DM']
    cols_to_puls = ['SN', 'DM', 'Sample']

    #Evaluo si en las columnas de los datos de presto, en las columnas especificadas, hay alguna fila que tenga el mismo valor que
    #en el dataframe a comparar, ya sea el de candidatos o el de pulsos

    #En el caso del dataframe de candidatos comparo solamente si hay pulsos en el dataframe de presto que tengan el mismo valor de DM que
    #aparece para los candidatos. Este criterio, va a dar 'True' si no hay nadie, por eso luego cuando creo la columna a agregar en el dataframe
    #lo niego, así es True cuando SÍ hay un candidato con ese valor de DM.
    no_match_results_cand = datatocompare[~datatocompare[cols_to_cand].apply(tuple, axis=1).isin(dfcand[cols_to_cand].apply(tuple, axis=1))]

    #El procedimiento para la comparación con el dataframe de pulsos es el mismo, solamente que en este caso, lo que comparo es la SN, el DM
    #y el número de Sample (creo que el número de sample ES CASI Q EL DNI).
    no_match_results_pul = datatocompare[~datatocompare[cols_to_puls].apply(tuple, axis=1).isin(dfpuls[cols_to_puls].apply(tuple, axis=1))]

    results_firstcom['Hay candi con = DM'] = ~results_firstcom.index.isin(no_match_results_cand.index)
    results_firstcom['Esta pulses'] = ~results_firstcom.index.isin(no_match_results_pul.index)

    
    return